In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def train_df(tr_path):
    classes, class_paths = zip(*[(label, os.path.join(tr_path, label, image))
                                 for label in os.listdir(tr_path) if os.path.isdir(os.path.join(tr_path, label))
                                 for image in os.listdir(os.path.join(tr_path, label))])

    tr_df = pd.DataFrame({'Class Path': class_paths, 'Class': classes})
    return tr_df

In [ ]:
def test_df(ts_path):
    classes, class_paths = zip(*[(label, os.path.join(ts_path, label, image))
                                 for label in os.listdir(ts_path) if os.path.isdir(os.path.join(ts_path, label))
                                 for image in os.listdir(os.path.join(ts_path, label))])

    ts_df = pd.DataFrame({'Class Path': class_paths, 'Class': classes})
    return ts_df

In [ ]:
tr_df = train_df('/kaggle/input/brain-tumor-mri-dataset/Training')

In [ ]:
ts_df = test_df('/kaggle/input/brain-tumor-mri-dataset/Testing')

In [ ]:
# Count of images in each class in train data
plt.figure(figsize=(15,7))
ax = sns.countplot(data=tr_df , y=tr_df['Class'])

plt.xlabel('')
plt.ylabel('')
plt.title('Count of images in each class', fontsize=20)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
#Count each class in test data
plt.figure(figsize=(15, 7))
ax = sns.countplot(y=ts_df['Class'], palette='viridis')

ax.set(xlabel='', ylabel='', title='Count of images in each class')
ax.bar_label(ax.containers[0])

plt.show()

In [ ]:
valid_df, ts_df = train_test_split(ts_df, train_size=0.5, random_state=20, stratify=ts_df['Class'])

In [ ]:
batch_size = 32
img_size = (299, 299)

_gen = ImageDataGenerator(rescale=1/255,
                          brightness_range=(0.8, 1.2))

ts_gen = ImageDataGenerator(rescale=1/255)


tr_gen = _gen.flow_from_dataframe(tr_df, x_col='Class Path',
                                  y_col='Class', batch_size=batch_size,
                                  target_size=img_size)

valid_gen = _gen.flow_from_dataframe(valid_df, x_col='Class Path',
                                     y_col='Class', batch_size=batch_size,
                                     target_size=img_size)

ts_gen = ts_gen.flow_from_dataframe(ts_df, x_col='Class Path',
                                  y_col='Class', batch_size=16,
                                  target_size=img_size, shuffle=False)

In [ ]:
class_dict = tr_gen.class_indices
classes = list(class_dict.keys())
images, labels = next(ts_gen)

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(zip(images, labels)):
    plt.subplot(4,4, i + 1)
    plt.imshow(image)
    class_name = classes[np.argmax(label)]
    plt.title(class_name, color='k', fontsize=15)

plt.show()

In [ ]:
img_shape=(299,299,3)
base_model = tf.keras.applications.Xception(include_top= False, weights= "imagenet",
                            input_shape= img_shape, pooling= 'max')

# for layer in base_model.layers:
#     layer.trainable = False
    
model = Sequential([
    base_model,
    Flatten(),
    Dropout(rate= 0.3),
    Dense(128, activation= 'relu'),
    Dropout(rate= 0.25),
    Dense(4, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001),
              loss= 'categorical_crossentropy',
              metrics= ['accuracy',
                        Precision(),
                        Recall()])

model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
hist = model.fit(tr_gen,
                 epochs=10,
                 validation_data=valid_gen,
                 shuffle= False)

In [ ]:
hist.history.keys()

In [ ]:
tr_acc = hist.history['accuracy']
tr_loss = hist.history['loss']
tr_per = hist.history['precision']
tr_recall = hist.history['recall']
val_acc = hist.history['val_accuracy']
val_loss = hist.history['val_loss']
val_per = hist.history['val_precision']
val_recall = hist.history['val_recall']

index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
index_precision = np.argmax(val_per)
per_highest = val_per[index_precision]
index_recall = np.argmax(val_recall)
recall_highest = val_recall[index_recall]

Epochs = [i + 1 for i in range(len(tr_acc))]
loss_label = f'Best epoch = {str(index_loss + 1)}'
acc_label = f'Best epoch = {str(index_acc + 1)}'
per_label = f'Best epoch = {str(index_precision + 1)}'
recall_label = f'Best epoch = {str(index_recall + 1)}'


plt.figure(figsize=(20, 12))
plt.style.use('fivethirtyeight')


plt.subplot(2, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label='Training loss')
plt.plot(Epochs, val_loss, 'g', label='Validation loss')
plt.scatter(index_loss + 1, val_lowest, s=150, c='blue', label=loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label='Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label='Validation Accuracy')
plt.scatter(index_acc + 1, acc_highest, s=150, c='blue', label=acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 3)
plt.plot(Epochs, tr_per, 'r', label='Precision')
plt.plot(Epochs, val_per, 'g', label='Validation Precision')
plt.scatter(index_precision + 1, per_highest, s=150, c='blue', label=per_label)
plt.title('Precision and Validation Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 4)
plt.plot(Epochs, tr_recall, 'r', label='Recall')
plt.plot(Epochs, val_recall, 'g', label='Validation Recall')
plt.scatter(index_recall + 1, recall_highest, s=150, c='blue', label=recall_label)
plt.title('Recall and Validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.show()

In [ ]:
train_score = model.evaluate(tr_gen, verbose=1)
valid_score = model.evaluate(valid_gen, verbose=1)
test_score = model.evaluate(ts_gen, verbose=1)

print(f"Train Loss: {train_score[0]:.4f}")
print(f"Train Accuracy: {train_score[1]*100:.2f}%")
print('-' * 20)
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"Validation Accuracy: {valid_score[1]*100:.2f}%")
print('-' * 20)
print(f"Test Loss: {test_score[0]:.4f}")
print(f"Test Accuracy: {test_score[1]*100:.2f}%")

In [ ]:
preds = model.predict(ts_gen)
y_pred = np.argmax(preds, axis=1)

In [ ]:
cm = confusion_matrix(ts_gen.classes, y_pred)
labels = list(class_dict.keys())
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('Truth Label')
plt.show()


In [ ]:
clr = classification_report(ts_gen.classes, y_pred)
print(clr)

In [ ]:
def predict(img_path):
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    label = list(class_dict.keys())
    plt.figure(figsize=(12, 12))
    img = Image.open(img_path)
    resized_img = img.resize((299, 299))
    img = np.asarray(resized_img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    predictions = model.predict(img)
    probs = list(predictions[0])
    labels = label
    plt.subplot(2, 1, 1)
    plt.imshow(resized_img)
    plt.subplot(2, 1, 2)
    bars = plt.barh(labels, probs)
    plt.xlabel('Probability', fontsize=15)
    ax = plt.gca()
    ax.bar_label(bars, fmt = '%.2f')
    plt.show()

In [ ]:
predict('/kaggle/input/brain-tumor-mri-dataset/Training/meningioma/Tr-me_0019.jpg')

In [ ]:

# Save the model
torch.save(model.state_dict(), 'medical_classification_model.pth')


In [ ]:
model.save('medical_classification_model.keras')

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Load the saved Keras model
model = keras.models.load_model('/kaggle/working/medical_classification_model.keras')

# Check the model architecture
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Load the saved Keras model
model = keras.models.load_model('/kaggle/working/medical_classification_model.keras')

# Check the model architecture
model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from PIL import Image
import os

# Load the saved Keras model
model = keras.models.load_model('medical_classification_model.keras')

# Specify the directory containing images
image_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/'

# List files in the directory
image_files = os.listdir(image_dir)

# Ensure there are images in the directory
if not image_files:
    raise FileNotFoundError("No images found in the specified directory!")

# Pick the first image from the directory
image_path = os.path.join(image_dir, image_files[0])

# Load the image
image = Image.open(image_path)

# Preprocessing (matching expected input size)
preprocess = keras.Sequential([
    keras.layers.Resizing(299, 299),  # ✅ Resize to match model's expected input size
    keras.layers.Rescaling(1./255)    # Normalize pixel values to [0,1]
])

# Convert the image to tensor
image = np.array(image)
image = preprocess(tf.convert_to_tensor(image))
image = tf.expand_dims(image, axis=0)  # Add batch dimension

# Perform inference
predictions = model.predict(image)
predicted_class = np.argmax(predictions, axis=1)[0]

# Map the predicted class to the class name
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Match training classes
predicted_class_name = class_names[predicted_class]

print(f'The predicted class is: {predicted_class_name}')

# Display the image with the predicted class name
plt.imshow(np.array(Image.open(image_path)))  # Reload for display
plt.axis('off')
plt.text(10, 10, f'Predicted: {predicted_class_name}', fontsize=12, color='white', backgroundcolor='red')
plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from PIL import Image
import os

# Load the saved Keras model
model = keras.models.load_model('medical_classification_model.keras')

# Specify the directory containing images
image_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/'

# List valid image files
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Ensure there are images in the directory
if not image_files:
    raise FileNotFoundError("No images found in the specified directory!")

# Define preprocessing function
preprocess = keras.Sequential([
    keras.layers.Resizing(299, 299),  # Resize to match model's expected input size
    keras.layers.Rescaling(1./255)    # Normalize pixel values to [0,1]
])

# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Match training classes

# Process and predict for each image
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    
    # Load image and ensure RGB mode
    image = Image.open(image_path).convert('RGB')  # ✅ Convert grayscale images to RGB
    
    # Convert the image to tensor and preprocess
    image_tensor = preprocess(tf.convert_to_tensor(np.array(image)))
    image_tensor = tf.expand_dims(image_tensor, axis=0)  # Add batch dimension

    # Perform inference
    predictions = model.predict(image_tensor)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class]

    # Print the result
    print(f'Image: {image_file} --> Predicted: {predicted_class_name}')

    # Display the image with prediction
    plt.figure()
    plt.imshow(np.array(image))  # Display image
    plt.axis('off')
    plt.text(10, 10, f'Predicted: {predicted_class_name}', fontsize=12, color='white', backgroundcolor='red')
    plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from PIL import Image
import os
from collections import defaultdict

# Load the saved Keras model
model = keras.models.load_model('medical_classification_model.keras')

# Specify the directory containing images
image_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/'

# List valid image files
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Ensure there are images in the directory
if not image_files:
    raise FileNotFoundError("No images found in the specified directory!")

# Define preprocessing function
preprocess = keras.Sequential([
    keras.layers.Resizing(299, 299),  # Resize to match model's expected input size
    keras.layers.Rescaling(1./255)    # Normalize pixel values to [0,1]
])

# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Match training classes

# Dictionary to count occurrences of each class
class_counts = defaultdict(int)

# Process and predict for each image
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    
    # Load image and ensure RGB mode
    image = Image.open(image_path).convert('RGB')  # ✅ Convert grayscale images to RGB
    
    # Convert the image to tensor and preprocess
    image_tensor = preprocess(tf.convert_to_tensor(np.array(image)))
    image_tensor = tf.expand_dims(image_tensor, axis=0)  # Add batch dimension

    # Perform inference
    predictions = model.predict(image_tensor)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class]

    # Update class count
    class_counts[predicted_class_name] += 1

    # Print the result
    print(f'Image: {image_file} --> Predicted: {predicted_class_name}')

    # Display the image with prediction
    plt.figure()
    plt.imshow(np.array(image))  # Display image
    plt.axis('off')
    plt.text(10, 10, f'Predicted: {predicted_class_name}', fontsize=12, color='white', backgroundcolor='red')
    plt.show()

# Print total number of images per class
print("\n=== Total Number of Images per Class ===")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")
